In [7]:
import pandas as pd

file_path = 'C:/Users/moura/PAD_JOB/data_extracted/Dados-BigQuery-comentarios-traduzidos.csv'
dataset = pd.read_csv(file_path)


dataset.head()

,int64_field_0,searchString,rank,isAdvertisement,title,price,categoryName,address,neighborhood,street,...,reviewsCount,scrapedAt,openingHours,peopleAlsoSearch,reviewsTags,additionalInfo,questionsAndAnswers,reviews,texto,texto_traduzido
0,335,bares,19,False,JOE'S Chopperia,NaN,Chop bar,"Avenida Milão Qd. 2, Lt. 5, Número: 1163 Celin...","Avenida Milão Qd. 2, Lt. 5, Número: 1163 Celin...","Avenida Milão Qd. 2, Lt. 5, Número: 1163 Celin...",...,1,2023-07-28 23:33:39.724000+00:00,"[{'day': 'sexta-feira', 'hours': '11:30 to 14:...","[{'category': 'As pessoas também pesquisaram',...",[],{'Da empresa': [{'Se identifica como uma empre...,NaN,"{'name': 'NATTHAN FOX', 'text': 'Top demais', ...",Top demais,Too top
1,471,bares,12,False,Bar e Lanchonete Cruzeiro - Setor novo horizonte,$$,Bar,"Av. César Lattes, 656 - Vila Novo Horizonte, G...","Av. César Lattes, 656","Av. César Lattes, 656",...,772,2023-07-28 23:58:29.022000+00:00,"[{'day': 'sexta-feira', 'hours': '10:00 to 00:...","[{'category': 'As pessoas também pesquisaram',...","[{'title': 'frango assado', 'count': 81}, {'ti...",{'Opções de serviço': [{'Entrega sem contato':...,{'question': 'Hj fiz um pedido nao foi nem 20 ...,"{'name': 'Natalice Liva', 'text': 'Minha exper...",Minha experiência no bar cruzeiro foi incrível...,"My experience at Bar Cruzeiro was amazing, the..."
2,471,bares,12,False,Bar e Lanchonete Cruzeiro - Setor novo horizonte,$$,Bar,"Av. César Lattes, 656 - Vila Novo Horizonte, G...","Av. César Lattes, 656","Av. César Lattes, 656",...,772,2023-07-28 23:58:29.022000+00:00,"[{'day': 'sexta-feira', 'hours': '10:00 to 00:...","[{'category': 'As pessoas também pesquisaram',...","[{'title': 'frango assado', 'count': 81}, {'ti...",{'Opções de serviço': [{'Entrega sem contato':...,{'question': 'Hj fiz um pedido nao foi nem 20 ...,"{'name': 'Karita Moreira', 'text': 'Minha expe...",Minha experiência foi com o caldo. Uma delícia...,"My experience was with the broth. A delight, v..."
3,471,bares,12,False,Bar e Lanchonete Cruzeiro - Setor novo horizonte,$$,Bar,"Av. César Lattes, 656 - Vila Novo Horizonte, G...","Av. César Lattes, 656","Av. César Lattes, 656",...,772,2023-07-28 23:58:29.022000+00:00,"[{'day': 'sexta-feira', 'hours': '10:00 to 00:...","[{'category': 'As pessoas também pesquisaram',...","[{'title': 'frango assado', 'count': 81}, {'ti...",{'Opções de serviço': [{'Entrega sem contato':...,{'question': 'Hj fiz um pedido nao foi nem 20 ...,"{'name': 'Iara Khabieva', 'text': 'Ambiente ab...","Ambiente aberto, passam bicicletas e pedestres...","Open environment, bicycles and pedestrians bet..."
4,471,bares,12,False,Bar e Lanchonete Cruzeiro - Setor novo horizonte,$$,Bar,"Av. César Lattes, 656 - Vila Novo Horizonte, G...","Av. César Lattes, 656","Av. César Lattes, 656",...,772,2023-07-28 23:58:29.022000+00:00,"[{'day': 'sexta-feira', 'hours': '10:00 to 00:...","[{'category': 'As pessoas também pesquisaram',...","[{'title': 'frango assado', 'count': 81}, {'ti...",{'Opções de serviço': [{'Entrega sem contato':...,{'question': 'Hj fiz um pedido nao foi nem 20 ...,"{'name': 'Cassia Mendes', 'text': 'O frango as...","O frango assado, o feijão tropeiro o melhor ai...","The roasted chicken, the tropeiro beans the be..."


In [8]:
dataset.iloc[2].at['texto']

'Minha experiência foi com o caldo. Uma delícia, muito bem temperado e bem feito! Recomendo! Um bar normal e tranquilo.\nCaldo de vaca atolada com muuuuita carne! Voltarei.'

In [9]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6865 entries, 0 to 6864
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   int64_field_0        6865 non-null   int64  
 1   searchString         6865 non-null   object 
 2   rank                 6865 non-null   int64  
 3   isAdvertisement      6865 non-null   bool   
 4   title                6865 non-null   object 
 5   price                5731 non-null   object 
 6   categoryName         6865 non-null   object 
 7   address              6865 non-null   object 
 8   neighborhood         6824 non-null   object 
 9   street               6824 non-null   object 
 10  city                 6865 non-null   object 
 11  postalCode           6790 non-null   object 
 12  state                6865 non-null   object 
 13  countryCode          6865 non-null   object 
 14  phone                6095 non-null   object 
 15  phoneUnformatted     6080 non-null   f

In [10]:
import pandas as pd

def map_sentiment(totalScore):
    if totalScore <= 2:
        return "negativo"
    elif totalScore == 3:
        return "neutro"
    else:
        return "positivo"
    
sentiment = dataset['totalScore'].apply(map_sentiment)


dataset = pd.DataFrame({
    'title' : dataset['title'],
    'categoryName': dataset['categoryName'],
    'postalCode' : dataset['postalCode'],
    'address' : dataset['address'],
    'reviews': dataset['reviews'],  
    'SENTIMENTO': sentiment,
    'totalScore': dataset['totalScore'],
    'texto': dataset['texto']
    #'TextBlob': dataset['TextBlob'],    
    #'texto_traduzido': dataset['texto_traduzido']
})



In [11]:
display(dataset)

,title,categoryName,postalCode,address,reviews,SENTIMENTO,totalScore,texto
0,JOE'S Chopperia,Chop bar,74373-270,"Avenida Milão Qd. 2, Lt. 5, Número: 1163 Celin...","{'name': 'NATTHAN FOX', 'text': 'Top demais', ...",positivo,5.0,Top demais
1,Bar e Lanchonete Cruzeiro - Setor novo horizonte,Bar,74363-010,"Av. César Lattes, 656 - Vila Novo Horizonte, G...","{'name': 'Natalice Liva', 'text': 'Minha exper...",positivo,4.3,Minha experiência no bar cruzeiro foi incrível...
2,Bar e Lanchonete Cruzeiro - Setor novo horizonte,Bar,74363-010,"Av. César Lattes, 656 - Vila Novo Horizonte, G...","{'name': 'Karita Moreira', 'text': 'Minha expe...",positivo,4.3,Minha experiência foi com o caldo. Uma delícia...
3,Bar e Lanchonete Cruzeiro - Setor novo horizonte,Bar,74363-010,"Av. César Lattes, 656 - Vila Novo Horizonte, G...","{'name': 'Iara Khabieva', 'text': 'Ambiente ab...",positivo,4.3,"Ambiente aberto, passam bicicletas e pedestres..."
4,Bar e Lanchonete Cruzeiro - Setor novo horizonte,Bar,74363-010,"Av. César Lattes, 656 - Vila Novo Horizonte, G...","{'name': 'Cassia Mendes', 'text': 'O frango as...",positivo,4.3,"O frango assado, o feijão tropeiro o melhor ai..."
...,...,...,...,...,...,...,...,...
6860,Boteco Raiz Marista,Bar,74160-010,"Av. 85, 1829 - St. Marista, Goiânia - GO, 7416...","{'name': 'KamiKazze3', 'text': 'tudo de bom, o...",positivo,4.0,"tudo de bom, o meu escritorio"
6861,Boteco Raiz Marista,Bar,74160-010,"Av. 85, 1829 - St. Marista, Goiânia - GO, 7416...","{'name': 'Luciano Carlos', 'text': 'Atendiment...",positivo,4.0,Atendimento Top
6862,Boteco Raiz Marista,Bar,74160-010,"Av. 85, 1829 - St. Marista, Goiânia - GO, 7416...","{'name': 'FOX BRAUSIM', 'text': 'Atendimento p...",positivo,4.0,Atendimento péssimo
6863,Boteco Raiz Marista,Bar,74160-010,"Av. 85, 1829 - St. Marista, Goiânia - GO, 7416...","{'name': 'Manoel Mauricio Carneiro Pereira', '...",positivo,4.0,Bom atendimento


Análise de Sentimentos por Comércio


In [12]:
# Agrupando o dataset por comércio e calculando a média da avaliação e da pontuação de sentimento (TextBlob)
analise_comercio = dataset.groupby('categoryName')[['totalScore']].mean()

# Ordenando os comércios pela média da avaliação
analise_comercio.sort_values('totalScore', ascending=False).head(10)


,totalScore
categoryName,
Bar com tabacaria,5.0
Chop bar,5.0
Quadra de esportes,4.9
Bar esportivo,4.8
Salão,4.8
Empresa de organização de eventos,4.7
Lanchonete,4.7
Cervejaria,4.7
Self service restaurant,4.7


In [13]:
# Agrupando o dataset por comércio e calculando a média da avaliação
analise_comercio_avaliacao = dataset.groupby('categoryName')['totalScore'].mean().reset_index()

# Ordenando os comércios pela média da avaliação
top_comercios = analise_comercio_avaliacao.sort_values('totalScore', ascending=False).head(5)
bottom_comercios = analise_comercio_avaliacao.sort_values('totalScore').head(5)

#top_comercios, bottom_comercios


In [14]:
print(top_comercios)

          categoryName  totalScore
2    Bar com tabacaria         5.0
12            Chop bar         5.0
16  Quadra de esportes         4.9
8        Bar esportivo         4.8
25               Salão         4.8


In [15]:
print(bottom_comercios)

                categoryName  totalScore
18    Restaurante brasileiro    4.246341
24           Saladas prontas    4.300000
4            Bar de narguilé    4.300000
23  Restaurante self-service    4.300000
14                 Gastropub    4.349630


In [21]:
# Selecionando comentários para o comércio com a maior avaliação média
comercio_maior_avaliacao = top_comercios.iloc[0]['categoryName']

comentarios_maior_avaliacao = dataset[dataset['categoryName'] == comercio_maior_avaliacao][['texto', 'totalScore']]



sample = comentarios_maior_avaliacao.sample(5, replace=True)
print(sample)

                                    texto  totalScore
32  A melhor boate da região.super indico         5.0
33       Super indico em qualquer ocasião         5.0
32  A melhor boate da região.super indico         5.0
32  A melhor boate da região.super indico         5.0
32  A melhor boate da região.super indico         5.0
